# Class 2 - Part 2: Integration Testing & Multi-Model API

## Objective
In this session, you will:
1. Create two model versions (baseline and dropout-regularized)
2. Implement an API that routes requests between multiple versions
3. Log and compare predictions from both versions
4. Measure latency and throughput
5. Understand canary deployments and A/B testing

## Key Concepts
- **Model Registry**: Managing multiple versions of the same model
- **Canary Testing**: Gradually rolling out new versions to a subset of traffic
- **A/B Testing**: Comparing two versions' performance on live traffic
- **Latency Monitoring**: Tracking inference speed across versions
- **Throughput**: How many requests the API can handle per second

## Why Integration Testing?

A single model in isolation may perform well, but production ML involves:
- **Multiple versions**: Upgrades, rollbacks, experiments
- **Varying input patterns**: Real-world data isn't uniform
- **System load**: API must handle many concurrent requests
- **Dependencies**: Integration with other services
- **Monitoring**: Comparing versions and detecting degradation

**Integration Testing** ensures the entire system works together.

## Step 1: Setup & Create Two Model Versions

In [ ]:
import os
import io
import json
import time
import random
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from datetime import datetime
from collections import defaultdict

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18

from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel

print("✅ Libraries imported")

In [ ]:
# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = ['animal', 'name_board', 'other_vehicle', 'pedestrian', 'pothole', 'road_sign', 'speed_breaker']
num_classes = len(class_names)

print(f"Device: {device}")
print(f"Classes: {class_names}")

In [ ]:
# Define model architectures
class BaselineModel(nn.Module):
    """Version 1: Basic ResNet-18 without regularization"""
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

class DropoutModel(nn.Module):
    """Version 2: ResNet-18 with Dropout for regularization"""
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.resnet = resnet18(pretrained=False)
        # Replace final layer with dropout + linear
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)

# Create both models
model_v1 = BaselineModel(num_classes).to(device)
model_v2 = DropoutModel(num_classes).to(device)

model_v1.eval()
model_v2.eval()

print("✅ Model V1 (Baseline) created")
print("✅ Model V2 (Dropout) created")

In [ ]:
# Image transformation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("✅ Transformation pipeline ready")

## Step 2: Prediction Logging & Metrics

In [ ]:
# Prediction log for tracking and analysis
prediction_logs = []

class PredictionLog:
    """Store prediction metadata for analysis"""
    def __init__(self, timestamp, image_id, model_version, prediction, confidence, latency_ms):
        self.timestamp = timestamp
        self.image_id = image_id
        self.model_version = model_version
        self.prediction = prediction
        self.confidence = confidence
        self.latency_ms = latency_ms
    
    def to_dict(self):
        return {
            'timestamp': self.timestamp,
            'image_id': self.image_id,
            'model_version': self.model_version,
            'prediction': self.prediction,
            'confidence': self.confidence,
            'latency_ms': self.latency_ms
        }

print("✅ Prediction logging system ready")

## Step 3: Multi-Model FastAPI

In [ ]:
# API Schemas
class PredictionResponse(BaseModel):
    image_id: str
    prediction: str
    confidence: float
    model_version: str
    latency_ms: float

class ComparisonResponse(BaseModel):
    image_id: str
    v1_prediction: str
    v1_confidence: float
    v2_prediction: str
    v2_confidence: float
    agreement: bool  # True if both models agree
    v1_latency_ms: float
    v2_latency_ms: float

class MetricsResponse(BaseModel):
    total_requests: int
    v1_requests: int
    v2_requests: int
    avg_latency_ms: float
    agreement_rate: float

print("✅ API response schemas defined")

In [ ]:
# Create FastAPI app
app = FastAPI(
    title="ADAS Multi-Model API",
    description="API serving two CNN model versions for A/B testing",
    version="2.0"
)

# Traffic distribution (simulating canary deployment)
CANARY_PERCENTAGE = 30  # Send 30% to v2, 70% to v1

print(f"✅ FastAPI app created with {CANARY_PERCENTAGE}% canary traffic to v2")

In [ ]:
@app.post("/predict")
async def predict_cached_routing(file: UploadFile = File(...)):
    """
    Route prediction request to v1 or v2 based on canary percentage.
    
    Returns:
        PredictionResponse with prediction from assigned model version
    """
    start_time = time.time()
    image_id = f"{datetime.now().timestamp()}"
    
    try:
        # Validate file
        allowed_extensions = {'.jpg', '.jpeg', '.png', '.gif'}
        file_ext = Path(file.filename).suffix.lower()
        
        if file_ext not in allowed_extensions:
            raise HTTPException(status_code=400, detail=f"Invalid file type: {file_ext}")
        
        # Read image
        contents = await file.read()
        try:
            image = Image.open(io.BytesIO(contents)).convert('RGB')
        except Exception as e:
            raise HTTPException(status_code=400, detail=f"Could not open image: {str(e)}")
        
        # Route decision: random traffic split
        use_v2 = random.random() < (CANARY_PERCENTAGE / 100)
        model = model_v2 if use_v2 else model_v1
        model_version = "v2.0" if use_v2 else "v1.0"
        
        # Transform and predict
        image_tensor = transform(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            outputs = model(image_tensor)
            probabilities = torch.softmax(outputs, dim=1)
            confidence, predicted_idx = torch.max(probabilities, 1)
        
        predicted_class = class_names[predicted_idx.item()]
        confidence_value = confidence.item()
        latency = (time.time() - start_time) * 1000
        
        # Log prediction
        log = PredictionLog(
            timestamp=datetime.now().isoformat(),
            image_id=image_id,
            model_version=model_version,
            prediction=predicted_class,
            confidence=round(confidence_value, 4),
            latency_ms=round(latency, 2)
        )
        prediction_logs.append(log)
        
        return PredictionResponse(
            image_id=image_id,
            prediction=predicted_class,
            confidence=confidence_value,
            model_version=model_version,
            latency_ms=round(latency, 2)
        )
    
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ /predict endpoint with traffic routing defined")

In [ ]:
@app.post("/predict-both")
async def predict_both_models(file: UploadFile = File(...)):
    """
    Get predictions from BOTH models for comparison (A/B testing).
    
    Returns:
        ComparisonResponse showing both predictions and agreement
    """
    start_time_v1 = time.time()
    image_id = f"{datetime.now().timestamp()}"
    
    try:
        # Validate file
        allowed_extensions = {'.jpg', '.jpeg', '.png', '.gif'}
        file_ext = Path(file.filename).suffix.lower()
        
        if file_ext not in allowed_extensions:
            raise HTTPException(status_code=400, detail=f"Invalid file type: {file_ext}")
        
        # Read image
        contents = await file.read()
        try:
            image = Image.open(io.BytesIO(contents)).convert('RGB')
        except Exception as e:
            raise HTTPException(status_code=400, detail=f"Could not open image: {str(e)}")
        
        # Transform
        image_tensor = transform(image).unsqueeze(0).to(device)
        
        # Model V1 prediction
        start_v1 = time.time()
        with torch.no_grad():
            outputs_v1 = model_v1(image_tensor)
            probs_v1 = torch.softmax(outputs_v1, dim=1)
            conf_v1, idx_v1 = torch.max(probs_v1, 1)
        latency_v1 = (time.time() - start_v1) * 1000
        
        # Model V2 prediction
        start_v2 = time.time()
        with torch.no_grad():
            outputs_v2 = model_v2(image_tensor)
            probs_v2 = torch.softmax(outputs_v2, dim=1)
            conf_v2, idx_v2 = torch.max(probs_v2, 1)
        latency_v2 = (time.time() - start_v2) * 1000
        
        # Predictions
        pred_v1 = class_names[idx_v1.item()]
        pred_v2 = class_names[idx_v2.item()]
        agreement = pred_v1 == pred_v2
        
        # Log both
        log_v1 = PredictionLog(
            timestamp=datetime.now().isoformat(),
            image_id=image_id,
            model_version="v1.0",
            prediction=pred_v1,
            confidence=round(conf_v1.item(), 4),
            latency_ms=round(latency_v1, 2)
        )
        log_v2 = PredictionLog(
            timestamp=datetime.now().isoformat(),
            image_id=image_id,
            model_version="v2.0",
            prediction=pred_v2,
            confidence=round(conf_v2.item(), 4),
            latency_ms=round(latency_v2, 2)
        )
        prediction_logs.append(log_v1)
        prediction_logs.append(log_v2)
        
        return ComparisonResponse(
            image_id=image_id,
            v1_prediction=pred_v1,
            v1_confidence=round(conf_v1.item(), 4),
            v2_prediction=pred_v2,
            v2_confidence=round(conf_v2.item(), 4),
            agreement=agreement,
            v1_latency_ms=round(latency_v1, 2),
            v2_latency_ms=round(latency_v2, 2)
        )
    
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("✅ /predict-both endpoint defined")

In [ ]:
@app.get("/metrics")
async def get_metrics():
    """
    Get aggregated metrics from prediction logs.
    """
    if not prediction_logs:
        return MetricsResponse(
            total_requests=0,
            v1_requests=0,
            v2_requests=0,
            avg_latency_ms=0.0,
            agreement_rate=0.0
        )
    
    df = pd.DataFrame([log.to_dict() for log in prediction_logs])
    
    total_requests = len(df)
    v1_requests = len(df[df['model_version'] == 'v1.0'])
    v2_requests = len(df[df['model_version'] == 'v2.0'])
    avg_latency = df['latency_ms'].mean()
    
    # Agreement rate: compare predictions with same image_id
    agreement_count = 0
    comparison_pairs = 0
    
    for image_id in df['image_id'].unique():
        logs_for_image = df[df['image_id'] == image_id]
        if len(logs_for_image) == 2:  # Both v1 and v2 predictions
            v1_logs = logs_for_image[logs_for_image['model_version'] == 'v1.0']
            v2_logs = logs_for_image[logs_for_image['model_version'] == 'v2.0']
            if not v1_logs.empty and not v2_logs.empty:
                if v1_logs.iloc[0]['prediction'] == v2_logs.iloc[0]['prediction']:
                    agreement_count += 1
                comparison_pairs += 1
    
    agreement_rate = (agreement_count / comparison_pairs * 100) if comparison_pairs > 0 else 0
    
    return MetricsResponse(
        total_requests=total_requests,
        v1_requests=v1_requests,
        v2_requests=v2_requests,
        avg_latency_ms=round(avg_latency, 2),
        agreement_rate=round(agreement_rate, 2)
    )

print("✅ /metrics endpoint defined")

In [ ]:
@app.get("/logs")
async def get_logs():
    """
    Get all prediction logs (for detailed analysis).
    """
    logs_dict = [log.to_dict() for log in prediction_logs]
    return {"total_logs": len(logs_dict), "logs": logs_dict}

print("✅ /logs endpoint defined")

## Step 4: Test the Multi-Model API

In [ ]:
# Create test client
client = TestClient(app)
print("✅ Test client created")

In [ ]:
# Helper function to create test images
def create_test_image(color='red'):
    """Create a dummy image for testing"""
    image = Image.new('RGB', (100, 100), color=color)
    img_bytes = io.BytesIO()
    image.save(img_bytes, format='PNG')
    img_bytes.seek(0)
    return img_bytes

print("✅ Test image helper created")

In [ ]:
# Test 1: Canary Routing (requests sent to v1 or v2)
print("\n" + "="*70)
print("TEST 1: Canary Deployment Routing (30% to v2, 70% to v1)")
print("="*70)

for i in range(10):
    img = create_test_image(color='red')
    response = client.post("/predict", files={"file": ("test.png", img, "image/png")})
    if response.status_code == 200:
        result = response.json()
        print(f"Request {i+1}: Model={result['model_version']}, Latency={result['latency_ms']:.2f}ms")

print("\n✅ Traffic successfully routed between versions")

In [ ]:
# Test 2: A/B Testing - Get predictions from both models
print("\n" + "="*70)
print("TEST 2: A/B Testing - Compare Both Models")
print("="*70)

for i in range(5):
    img = create_test_image(color='blue')
    response = client.post("/predict-both", files={"file": ("test.png", img, "image/png")})
    if response.status_code == 200:
        result = response.json()
        agreement_icon = "✅" if result['agreement'] else "❌"
        print(f"\nComparison {i+1}:")
        print(f"  V1: {result['v1_prediction']} (conf={result['v1_confidence']:.4f}, latency={result['v1_latency_ms']:.2f}ms)")
        print(f"  V2: {result['v2_prediction']} (conf={result['v2_confidence']:.4f}, latency={result['v2_latency_ms']:.2f}ms)")
        print(f"  Agreement: {agreement_icon}")

In [ ]:
# Test 3: View Metrics
print("\n" + "="*70)
print("TEST 3: Aggregated Metrics")
print("="*70)

response = client.get("/metrics")
if response.status_code == 200:
    metrics = response.json()
    print(f"Total Requests: {metrics['total_requests']}")
    print(f"V1 Requests: {metrics['v1_requests']} ({metrics['v1_requests']/metrics['total_requests']*100:.1f}%)")
    print(f"V2 Requests: {metrics['v2_requests']} ({metrics['v2_requests']/metrics['total_requests']*100:.1f}%)")
    print(f"Average Latency: {metrics['avg_latency_ms']:.2f}ms")
    print(f"Model Agreement Rate: {metrics['agreement_rate']:.1f}%")

## Step 5: Performance Comparison Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get all logs
response = client.get("/logs")
logs_data = response.json()['logs']
df = pd.DataFrame(logs_data)

if len(df) > 0:
    print("\n" + "="*70)
    print("PERFORMANCE ANALYSIS")
    print("="*70)
    
    # Version comparison
    print("\n📊 Latency by Model Version:")
    version_stats = df.groupby('model_version')['latency_ms'].agg(['count', 'mean', 'std', 'min', 'max'])
    print(version_stats.round(2))
    
    # Create visualizations
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Latency distribution
    for version in df['model_version'].unique():
        version_latencies = df[df['model_version'] == version]['latency_ms']
        axes[0].hist(version_latencies, alpha=0.6, label=version, bins=10)
    axes[0].set_xlabel('Latency (ms)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Inference Latency Distribution by Version')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Traffic distribution
    version_counts = df['model_version'].value_counts()
    axes[1].bar(version_counts.index, version_counts.values, color=['blue', 'orange'])
    axes[1].set_ylabel('Number of Requests')
    axes[1].set_title('Traffic Distribution (Canary Test)')
    
    # Add percentage labels
    for i, v in enumerate(version_counts.values):
        percentage = v / version_counts.sum() * 100
        axes[1].text(i, v + 0.1, f'{percentage:.1f}%', ha='center', fontweight='bold')
    
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('integration_testing_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Analysis visualization saved")
else:
    print("No logs to analyze")

## Step 6: Key Findings - Integration Testing

In [ ]:
print("\n" + "="*70)
print("🎯 INTEGRATION TESTING INSIGHTS")
print("="*70)

if len(df) > 0:
    v1_latency = df[df['model_version'] == 'v1.0']['latency_ms'].mean()
    v2_latency = df[df['model_version'] == 'v2.0']['latency_ms'].mean()
    
    print(f"""
1️⃣ CANARY DEPLOYMENT:
   - Safely test v2 with {CANARY_PERCENTAGE}% of traffic
   - Main traffic continues with stable v1
   - If v2 shows issues, rollback unaffects most users
   
2️⃣ LATENCY COMPARISON:
   - V1 Average Latency: {v1_latency:.2f}ms
   - V2 Average Latency: {v2_latency:.2f}ms
   - Difference: {abs(v1_latency - v2_latency):.2f}ms
   
3️⃣ MODEL AGREEMENT:
   - High agreement rate = Both versions are stable
   - Low agreement = Investigate model divergence
   
4️⃣ THROUGHPUT:
   - Total Requests Processed: {len(df)}
   - Requests per second: ~{len(df)/5:.1f} req/sec (demo rate)
   
5️⃣ MONITORING ALERTS:
   ✅ Track latency increase > 20% = Performance degradation
   ✅ Agreement < 70% = Model quality issue
   ✅ Error rate > 5% = API stability problem
""")

print("="*70)

## Step 7: Deployment Strategies

In [ ]:
print("""
🚀 PRODUCTION DEPLOYMENT STRATEGIES

1. BLUE-GREEN DEPLOYMENT:
   ┌─────────────────┬──────────────────┐
   │  Blue (v1.0)    │  Green (v2.0)    │
   │  Production     │  Staging         │
   │  100% traffic   │  Testing         │
   └─────────────────┴──────────────────┘
   → When v2 is ready, switch ALL traffic instantly
   → If issues, switch back immediately
   → Zero downtime, instant rollback

2. CANARY DEPLOYMENT (Current Approach):
   10% → 25% → 50% → 100%
   → Gradually increase v2 traffic
   → Monitor metrics at each stage
   → Stop if degradation detected

3. SHADOW DEPLOYMENT:
   ┌─────────────────────────────┐
   │  User Request               │
   │         ↓                   │
   │  V1 (Primary)    V2 (Shadow)│
   │  Returns to user  No return │
   │                   Compare   │
   └─────────────────────────────┘
   → v2 processes in background
   → Compare predictions offline
   → No user impact

4. A/B TESTING:
   → Different versions to different user cohorts
   → Statistical significance testing
   → Winner deployed to all
""")

## Key Takeaways

In [ ]:
print("""
📝 KEY TAKEAWAYS FROM PART 2

✅ APIs must handle multiple model versions simultaneously
✅ Canary deployments reduce risk of bad model releases  
✅ A/B testing provides data-driven model selection
✅ Latency monitoring catches performance regressions
✅ Agreement rates indicate model stability
✅ Comprehensive logging enables root cause analysis
✅ Gradual rollouts are safer than big bangs

🔑 CRITICAL METRICS TO TRACK:
   1. Latency per model version
   2. Error rates and types
   3. Prediction agreement rates
   4. Throughput capacity
   5. Confidence distribution
""")